In [ ]:
%%html
<style>   
div.input {   
    display:none;   
}
/style>

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import Text, Div, ColumnDataSource, Range1d
from bokeh.models import RadioButtonGroup, CustomJS
from bokeh.layouts import gridplot, row
from threading import Thread
import numpy as np
import time

In [ ]:
from pat20 import pat
from i2c_client import i2c
from sysmon_client import sysmon

In [ ]:
# Output Bokeh plots to the notebook
output_notebook()

# Initalize the numpy arrays for taking the power values
HIST_VALUES = 60
x_fpd = np.arange(-HIST_VALUES, 1, 1.0,  dtype=float)
y_fpd = np.zeros(HIST_VALUES+1, dtype=float)

x_lpd = np.arange(-HIST_VALUES, 1, 1.0,  dtype=float)
y_lpd = np.zeros(HIST_VALUES+1, dtype=float)

x_pld = np.arange(-HIST_VALUES, 1, 1.0,  dtype=float)
y_pld = np.zeros(HIST_VALUES+1, dtype=float)

x_full = np.arange(-HIST_VALUES, 1, 1.0,  dtype=float)
y_full = np.zeros(HIST_VALUES+1, dtype=float)


In [ ]:
# Setup the communication through RAFT and initialize i2c
i2c.SetIpAndPort("127.0.0.1", 9090)
sysmon.SetIpAndPort("127.0.0.1", 9090)
ret = i2c.XI2c_Initialize(2)

In [ ]:
# Create a Div widget for displaying the title
header_html="""<center><h2 style='font-size: 35pt;margin-top: 10px; 
                margin-left: 200px;'>Power Advantage Tool</h2></center>"""
heading = Div(text=header_html, width=750, height=100)

# Create a rectangular bar chart for displaying the power values
bar_names = ["FPD", "LPD", "PLD", "TOT"]
colors = ["blue", "red", "green", "black"]
tools_to_include = []
bar_fig = figure(width=200, height=200, title="", x_range=bar_names, tools=tools_to_include)
bars = bar_fig.vbar(x=bar_names, top=[0, 0, 0, 0], width=0.5, color=colors)

# Create a text block using Div for the left column
left_text_block = Div(text="""<span style='font-size: 12pt; text-align: center;'>
                      <b>Initial values:</b>""", width=300, height=200)

# Create a text block using Div for the right column
right_text_block = Div(text="", width=300, height=200)

# Create a figure
p = figure(width=800, height=750, title="", x_axis_label="secs", y_axis_label="mW", tools=tools_to_include)
p.x_range = Range1d(-HIST_VALUES, 10)

# Create the HTML Divs for the two pages
sysmon_div = Div(text="""
<div style="border: 2px solid #007BFF; border-radius: 10px; padding: 10px; text-align: center;">
    <p><strong>Page 1:</strong> Initial content for Page 1. Value: <span id="sysmon_value">0.00</span></p>
</div>
""", width=750)

rails_div = Div(text="""
<div style="border: 2px solid #E83E8C; border-radius: 10px; padding: 10px; text-align: center;">
    <p><strong>Page 2:</strong> Initial content for Page 2. Value: <span id="rails_value">0.00</span></p>
</div>
""", width=750)

# Create RadioButtonGroup widget
radio_button_group = RadioButtonGroup(labels=["GRAPH", "SYSMON", "RAILS"], active=0)


In [ ]:
# Create the the power values in the Graph
line_fpd = p.line(x_fpd, y_fpd, line_width=2, legend_label="FPD", line_color="blue")
line_lpd = p.line(x_lpd, y_lpd, line_width=2, legend_label="LPD", line_color="red")
line_pld = p.line(x_pld, y_pld, line_width=2, legend_label="PLD", line_color="green")
line_full = p.line(x_full, y_full, line_width=2, legend_label="TOTAL", line_color="black")

# Position the legend on the left corner
p.legend.location = "top_left"

# Create Text glyphs for each line's text indicator
text_source_fpd = ColumnDataSource(data=dict(x=[x_fpd[-1]], y=[y_fpd[-1]], text=[f'..FPD {y_fpd[-1]:.2f}']))
text_source_lpd = ColumnDataSource(data=dict(x=[x_lpd[-1]], y=[y_lpd[-1]], text=[f'..LPD {y_lpd[-1]:.2f}']))
text_source_pld = ColumnDataSource(data=dict(x=[x_pld[-1]], y=[y_pld[-1]], text=[f'..PLD {y_pld[-1]:.2f}']))
text_source_full = ColumnDataSource(data=dict(x=[x_full[-1]], y=[y_full[-1]], text=[f'..TOTAL {y_full[-1]:.2f}']))

# Add the text glyphs to the figure
text_fpd = Text(x='x', y='y', text='text', text_color='blue', text_font_size='10pt')
p.add_glyph(text_source_fpd, text_fpd);

text_lpd = Text(x='x', y='y', text='text', text_color='red', text_font_size='10pt')
p.add_glyph(text_source_lpd, text_lpd);

text_pld = Text(x='x', y='y', text='text', text_color='green', text_font_size='10pt')
p.add_glyph(text_source_pld, text_pld);

text_full = Text(x='x', y='y', text='text', text_color='black', text_font_size='10pt')
p.add_glyph(text_source_full, text_full);


In [ ]:
# Create CustomJS callback
callback = CustomJS(args={
    'p': p,
    'sysmon_div': sysmon_div,
    'rails_div': rails_div,
    'radio_button_group': radio_button_group,
}, code="""
const choice = radio_button_group.active;
if (choice === 0) {
    p.visible = true;
    sysmon_div.visible = false;
    rails_div.visible = false;
} else if (choice === 1) {
    p.visible = false;
    sysmon_div.visible = true;
    rails_div.visible = false;
} else if (choice === 2) {
    p.visible = false;
    sysmon_div.visible = false;
    rails_div.visible = true;
}
""")

# Attach the callback to the RadioButtonGroup
radio_button_group.js_on_change('active', callback)

# Set initial visibility
sysmon_div.visible = False
rails_div.visible = False

# Use gridplot to arrange the plots
grid = gridplot([[heading],[row(bar_fig, left_text_block, right_text_block)], [radio_button_group], 
                 [row(p, sysmon_div, rails_div)]], toolbar_options={'logo': None})

# Display the initial plot
plot_handle = show(grid, notebook_handle=True)

In [ ]:
def update():
    global x_fpd
    global y_fpd
    global x_lpd
    global y_lpd
    global x_pld
    global y_pld
    global x_full
    global y_full
    global bars
    while True:
        # Take the values to be updated
        pat.PullData(i2c, sysmon)
        str0 = pat.generateFullReport()
        str1 = pat.generateFullSysmonReport()
        y_fpd = np.append(y_fpd, pat.fpdPower/1000)
        if(y_fpd.shape[0] > HIST_VALUES+1):
            y_fpd = y_fpd[1:HIST_VALUES+2]
        y_lpd = np.append(y_lpd, pat.lpdPower/1000)
        if(y_lpd.shape[0] > HIST_VALUES+1):
            y_lpd = y_lpd[1:HIST_VALUES+2]
        y_pld = np.append(y_pld, pat.pldPower/1000)
        if(y_pld.shape[0] > HIST_VALUES+1):
            y_pld = y_pld[1:HIST_VALUES+2]
        y_full = np.append(y_full, pat.fpdPower/1000 + pat.lpdPower/1000 + pat.pldPower/1000)
        if(y_full.shape[0] > HIST_VALUES+1):
            y_full = y_full[1:HIST_VALUES+2]

        # Update the rectangular bar chart
        bars_data = [int(np.round(y_fpd[-1],2)), int(np.round(y_pld[-1],2)), 
                     int(np.round(y_lpd[-1],2)), int(np.round(y_full[-1],2))]
        bars.data_source.data["top"] = bars_data        
        
        # Update left column text block with power values
        fpd_dstr = f"FPD: {np.round(y_fpd[-1],2)} (mW)"
        pld_dstr = f"PLD: {np.round(y_pld[-1],2)} (mW)"
        lpd_dstr = f"LPD: {np.round(y_lpd[-1],2)} (mW)"
        full_dstr = f"TOT: {np.round(y_full[-1],2)} (mW)"        
        left_new_text = f"""
        <p>{fpd_dstr}</p>
        <p>{pld_dstr}</p>
        <p>{lpd_dstr}</p>
        <p>{full_dstr}</p>
        <span style='font-size: 12pt; text-align: center;'>"""        
        left_text_block.text = left_new_text

        # Update the PS temperature
        tempStr = "PS {0:3.1f}° C".format(float(pat.psTemp))
        tempLabel = "<h1 style='font-size:40px'>" + tempStr +  "</h1>"
        right_new_text = tempLabel
        right_text_block.text = right_new_text        
                
        # Update the line graph
        line_fpd.data_source.data["y"] = y_fpd
        line_lpd.data_source.data["y"] = y_lpd
        line_pld.data_source.data["y"] = y_pld
        line_full.data_source.data["y"] = y_full
        text_source_fpd.data = dict(x=[x_fpd[-1]], y=[y_fpd[-1]], text=[f'..FPD {y_fpd[-1]:.2f}'])
        text_source_lpd.data = dict(x=[x_lpd[-1]], y=[y_lpd[-1]], text=[f'..LPD {y_lpd[-1]:.2f}'])
        text_source_pld.data = dict(x=[x_pld[-1]], y=[y_pld[-1]], text=[f'..PLD{y_pld[-1]:.2f}'])  
        text_source_full.data = dict(x=[x_full[-1]], y=[y_full[-1]], text=[f'..TOTAL{y_full[-1]:.2f}'])

        # Update the sysmon and rails tab
        sysmon_div.text = str0
        rails_div.text = str1
       
        # Update and refresh bokeh plot every second
        push_notebook(handle=plot_handle)
        time.sleep(1)  

In [ ]:
# Create a thread to update the line data
update_thread = Thread(target=update)
update_thread.daemon = True
update_thread.start()

# Copyright (C) 2021-2022 Xilinx, Inc.  All rights reserved.
# Copyright (C) 2022-2023 Advanced Micro Devices, Inc. All Rights Reserved.
# SPDX-License-Identifier: BSD-3-Clause